# Trouver un compromis entre une valeur de pureté élevée et un nombre d'élements de signal sufisament important 

In [2]:
import pandas
import uproot
import matplotlib.pyplot as plt
import numpy as np
import scipy.interpolate as interpolate
import pickle

from matplotlib import rc
rc('font',**{'family':'serif','serif':['Roman']})
rc('text', usetex=True)
params= {'text.latex.preamble' : [r'\usepackage{amsmath}']}
plt.rcParams.update(params)
plt.rcParams['text.latex.preamble'] = r"\usepackage{bm} \usepackage{amsmath}"

/cvmfs/sft.cern.ch/lcg/releases/Python/3.8.6-3199b/x86_64-centos7-gcc8-opt/lib/python3.8/_collections_abc.py:832: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.
  self[key] = other[key]


In [3]:
bc_tree = uproot.open('/eos/experiment/fcc/ee/analyses/case-studies/flavour/Bc2TauNu/flatNtuples/spring2021/prod_04/Analysis_stage2/p8_ee_Zbb_ecm91_EvtGen_Bc2TauNuTAUHADNU.root')['events']
bb_tree = uproot.open('/eos/experiment/fcc/ee/analyses/case-studies/flavour/Bc2TauNu/flatNtuples/spring2021/prod_04/Analysis_stage2/p8_ee_Zbb_ecm91_EvtGen.root')['events']
bu_tree = uproot.open('/eos/experiment/fcc/ee/analyses/case-studies/flavour/Bc2TauNu/flatNtuples/spring2021/prod_04/Analysis_stage2/p8_ee_Zbb_ecm91_EvtGen_Bu2TauNuTAUHADNU.root')['events']
tree_gen = uproot.open("/eos/experiment/fcc/ee/analyses/case-studies/flavour/Bc2TauNu/flatNtuples/spring2021/prod_04/Analysis_stage2/p8_ee_Zbb_ecm91_EvtGen_Bc2TauNuTAUHADNU.root")["metadata"]

In [ ]:
bc_df = bc_tree.arrays(library="pd", how="zip", filter_name=["EVT_*"])
bb_df = bb_tree.arrays(library="pd", how="zip", filter_name=["EVT_*"])
bu_df = bu_tree.arrays(library="pd", how="zip", filter_name=["EVT_*"])
df_gen = tree_gen.arrays(library="pd", how="zip")

In [4]:
N_Z0 = 5*10**(12)
BF_Zbb = 0.1512
f_Bc = 0.0004
BF_Bctaunu = 0.0194
BF_tau3pion = 0.0931

In [5]:
def S_cut(initial_eff_signal, cut_eff) :
    return N_Z0*BF_Zbb*2*f_Bc*BF_Bctaunu*BF_tau3pion*initial_eff_signal*cut_eff

def B_incl_cut(initial_eff_incl_bkg, cut_eff) :
    return N_Z0*BF_Zbb*2*initial_eff_incl_bkg*cut_eff

In [6]:
spline_MVA1_log = pickle.load(open('Spline_functions/MVA1_spline_log.pkl','rb'))
spline_MVA2_log = pickle.load(open('Spline_functions/MVA2_spline_log.pkl','rb'))

# Superposition plot imshow purity et scatter du nombre d'évènements de signal

cut_min = 0.995, cut_step = 0.0001 --> bcp points pour scatter + imshow

cut_min = 0.999, cut_step = 0.00007

cut_min = 0.999, cut_step = 0.0005 

In [ ]:
cut_step = 0.00005
#100 valeurs donne un effet continu

#MVA1
var1 = 'EVT_MVA1'
min_cut_1 = 0.995
max_cut_1_log = 9

min_cut_1_log = -np.log(1 - min_cut_1)
max_cut_1 = 1 - np.exp(-max_cut_1_log)
nb_bkg_events_1 = interpolate.splint(0.95, 9, spline_MVA1_log) # Valeurs extrémales def spline

#MVA2
var2 = 'EVT_MVA2'
min_cut_2 = 0.995
max_cut_2_log = 8

max_cut_2 = 1 - np.exp(-max_cut_2_log)
min_cut_2_log = -np.log(1 - min_cut_2)
nb_bkg_events_2 = interpolate.splint(0.95, 8, spline_MVA2_log) # Valeurs extrémales def spline

#Parameters
sig = bc_df[bc_df[var2]> 0.95]
sig = sig[sig[var1]> 0.95]
bkg = bb_df[bb_df[var2]> 0.95]  #Il faut que l'on considère le même cut de départ pour tous
bkg = bkg[bkg[var1]> 0.95] # (pour les len ou pour les splines)

nb_total_events = df_gen.iloc[0]["eventsProcessed"]
nb_signal_events = len(sig)
nb_bkg_events = len(bkg)

initial_eff_signal = nb_signal_events / nb_total_events
initial_eff_incl_bkg = nb_bkg_events / nb_total_events

#print('eff sig {}'.format(initial_eff_signal))
#print('eff bkg {}'.format(initial_eff_incl_bkg))

#np.array()
cut_1 = np.arange(min_cut_1, max_cut_1, cut_step)
cut_2 = np.arange(min_cut_2, max_cut_2, cut_step)
p_tab = np.zeros((np.size(cut_2),np.size(cut_1)))
s_tab = np.zeros((np.size(cut_2),np.size(cut_1)))
b_tab = np.zeros((np.size(cut_2),np.size(cut_1)))

print(cut_1)
print(cut_2)

#To help find a compromise between the purity and the number of signal events
N_sig = 4000
mva1 = []
mva2 = []
N_sig_list = []


for i1, cut_i1 in enumerate(cut_1) :
    
    cut_i1_log = -np.log(1 - cut_i1)
    df_sig = bc_df[ bc_df[var1] > cut_i1 ]
    
    for i2, cut_i2 in enumerate(cut_2) :

        cut_i2_log = -np.log(1-cut_i2) #log
        df_signal = df_sig[df_sig[var2]>cut_i2]

        nb_sig_cut_i1_i2 = len(df_signal)
        nb_bkg_cut_i1 = interpolate.splint(cut_i1_log, max_cut_1_log, spline_MVA1_log) #log
        nb_bkg_cut_i2 = interpolate.splint(cut_i2_log, max_cut_2_log, spline_MVA2_log) #log
        
        signal_cut_eff = nb_sig_cut_i1_i2 / nb_signal_events
        bkg_cut_eff = (nb_bkg_cut_i1 / nb_bkg_events_1) * (nb_bkg_cut_i2 / nb_bkg_events_2)

        s = S_cut(initial_eff_signal, signal_cut_eff)
        b = B_incl_cut(initial_eff_incl_bkg, bkg_cut_eff) 
        
        if b==0 :
            print('B(MVA1,MVA2)=0 pour le cut MVA1 = {} et pour le cut MVA2 = {} ce qui correspond à la coordonées ({},{})'.format(cut_i1,cut_i2,i2,i1))
            ##### Vérification qu'il n'y a pas de valeurs de background égales à 0
            
        s_tab[i2,i1] = s
        b_tab[i2,i1] = b
        p_tab[i2,i1] = s/(s+b)

        mva1.append(i1)
        mva2.append(i2)
        N_sig_list.append(nb_sig_cut_i1_i2)

In [ ]:
precision = 4

plt.figure(figsize=(25,8))

nb_sig = plt.scatter(mva1, mva2, c=N_sig_list, cmap='plasma_r', s=20, label='nb sig events $>$ {}'.format(N_sig))
cb_nb_sig = plt.colorbar(nb_sig, fraction=0.0345)
cb_nb_sig.ax.tick_params(labelsize=15)
cb_nb_sig.set_label('Number of signal events', fontsize=20, labelpad=10) #$>$ {}'.format(N_sig)

purity = plt.imshow(p_tab, cmap='Blues', origin='lower')
cb_purity = plt.colorbar(purity, fraction=0.0355)
cb_purity.ax.tick_params(labelsize=15)
cb_purity.set_label('P(MVA1,MVA2)', fontsize=20, labelpad=15)

plt.tick_params(axis='both', which='major', labelsize=15)

plt.xticks(np.arange(len(cut_1)),np.around(cut_1,precision), rotation=60)
plt.yticks(np.arange(len(cut_2)),np.around(cut_2,precision))
plt.xlabel('Value of the MVA1 cut', fontsize=20, labelpad=20)
plt.ylabel('Value of the MVA2 cut', fontsize=20, labelpad=20)

plt.show()

In [ ]:
plt.figure()
plt.imshow(s_tab, cmap='Orangess', origin='lower', extent=(cut_1[0], cut_1[-1], cut_2[0], cut_2[-1])) #(vmin=0, vmax=0.01)
plt.xlabel('Value of the MVA1 cut')
plt.ylabel('Value of the MVA2 cut')
plt.title('S(MVA1,MVA2)')
plt.xlim(cut_1[0],cut_1[-1])
plt.ylim(cut_2[0],cut_2[-1])
plt.colorbar()
plt.show()

In [ ]:
plt.figure()
plt.imshow(b_tab, cmap='Purples', origin='lower', extent=(cut_1[0], cut_1[-1], cut_2[0], cut_2[-1])) #(vmin=0, vmax=0.01)
plt.xlabel('Value of the MVA1 cut')
plt.ylabel('Value of the MVA2 cut')
plt.title('B(MVA1,MVA2)')
plt.xlim(cut_1[0],cut_1[-1])
plt.ylim(cut_2[0],cut_2[-1])
plt.colorbar()
plt.show()

# Déterminer les valeurs associées au meilleur compromis 

In [1]:
cut_step = 0.000005
#100 valeurs donnent un effet continu

#MVA1
var1 = 'EVT_MVA1'
min_cut_1 = 0.999
max_cut_1_log = 9

min_cut_1_log = -np.log(1 - min_cut_1)
max_cut_1 = 1 - np.exp(-max_cut_1_log)
nb_bkg_events_1 = interpolate.splint(0.95, 9, spline_MVA1_log) # Valeurs extrémales def spline

#MVA2
var2 = 'EVT_MVA2'
min_cut_2 = 0.996
max_cut_2_log = 8

max_cut_2 = 1 - np.exp(-max_cut_2_log)
min_cut_2_log = -np.log(1 - min_cut_2)
nb_bkg_events_2 = interpolate.splint(0.95, 8, spline_MVA2_log) # Valeurs extrémales def spline

#Parameters
sig = bc_df[bc_df[var2]> 0.95]
sig = sig[sig[var1]> 0.95]
bkg = bb_df[bb_df[var2]> 0.95]  #Il faut que l'on considère le même cut de départ pour tous
bkg = bkg[bkg[var1]> 0.95] # (pour les len ou pour les splines)

nb_total_events = df_gen.iloc[0]["eventsProcessed"]
nb_signal_events = len(sig)
nb_bkg_events = len(bkg)

initial_eff_signal = nb_signal_events / nb_total_events
initial_eff_incl_bkg = nb_bkg_events / nb_total_events

#print('eff sig {}'.format(initial_eff_signal))
#print('eff bkg {}'.format(initial_eff_incl_bkg))

#np.array()
cut_1_ = np.arange(min_cut_1, max_cut_1, cut_step)
cut_2_ = np.arange(min_cut_2, max_cut_2, cut_step)

Best_p = 0
Best_MVA1 = 0.95
Best_MVA2 = 0.95 #0.95 correspond à la valeur minimale des cuts
Best_n_sig = 0
Target_n_sig = 4000


for i1, cut_i1 in enumerate(cut_1_) :
   
    cut_i1_log = -np.log(1 - cut_i1)
    df_sig = bc_df[ bc_df[var1] > cut_i1 ]
    print(i1)
    
    for i2, cut_i2 in enumerate(cut_2_) :
        
        cut_i2_log = -np.log(1-cut_i2) #log
        df_signal = df_sig[df_sig[var2]>cut_i2]

        nb_sig_cut_i1_i2 = len(df_signal)
        nb_bkg_cut_i1 = interpolate.splint(cut_i1_log, max_cut_1_log, spline_MVA1_log) #log
        nb_bkg_cut_i2 = interpolate.splint(cut_i2_log, max_cut_2_log, spline_MVA2_log) #log
        
        signal_cut_eff = nb_sig_cut_i1_i2 / nb_signal_events
        bkg_cut_eff = (nb_bkg_cut_i1 / nb_bkg_events_1) * (nb_bkg_cut_i2 / nb_bkg_events_2)

        s = S_cut(initial_eff_signal, signal_cut_eff)
        b = B_incl_cut(initial_eff_incl_bkg, bkg_cut_eff) 
        p = s/(s+b)
        
        #if b==0 :
            #print('B(MVA1,MVA2)=0 pour le cut MVA1 = {} et pour le cut MVA2 = {} ce qui correspond à la coordonées ({},{})'.format(cut_i1,cut_i2,i2,i1))
            ##### Vérification qu'il n'y a pas de valeurs de background égales à 0
        
        if (s > Target_n_sig) & (p > Best_p) & (b>0) :
            
            Best_MVA1 = cut_i1
            Best_MVA2 = cut_i2
            Best_p = p
            Best_n_sig = s

    

NameError: name 'np' is not defined

In [ ]:
print('The best purity for a target of {} sig events is equal to = {}'.format(Target_n_sig, Best_p ))
print('It is associated to a MVA1 cut = {} and a MVA2 cut = {}'.format(Best_MVA1, Best_MVA2))
print('Total number of signal = {}'.format(Best_n_sig))

In [ ]:
print(len(cut_1))
print(cut_1)
print(len(cut_2))
print(cut_2)

##### Valeurs retenues pour les best cuts

cut_min = 0.995, cut1_max = 9, cut2_max = 8, cut_step=0.0005 -> The best purity for a target of 4000 sig events is equal to = 0.09334691238899527. It is associated to a MVA1 cut = 0.9994999999999995 and a MVA2 cut = 0.9984999999999996

cut_min = 0.995, cut1_max = 9, cut2_max = 8, cut_step=0.0001 -> The best purity for a target of 4000 sig events is equal to = 0.22525469254541738
It is associated to a MVA1 cut = 0.9997999999999995 and a MVA2 cut = 0.9986999999999996

cut_min = 0.995, cut1_max = 9, cut2_max = 8, cut_step=0.00009 --> The best purity for a target of 4000 sig events is equal to = 0.6079378810011188
It is associated to a MVA1 cut = 0.9998600000000019 and a MVA2 cut = 0.9986900000000014

cut_min = 0.995, cut1_max = 9, cut2_max = 8, cut_step=0.000085 --> The best purity for a target of 4000 sig events is equal to = 0.3929455591316377. It is associated to a MVA1 cut = 0.9998450000000001 and a MVA2 cut = 0.9986550000000001

The best purity for a target of 4000 sig events is equal to = 0.6079378810011188
It is associated to a MVA1 cut = 0.9998600000000019 and a MVA2 cut = 0.9986900000000014
Total number of signal = 5999

    $B$ = $N(Z^0) \times \mathscr{B}(Z^0 \rightarrow b\bar{b}) \times 2 \times f(B^+) \times \mathscr{B}(B^+ \rightarrow \tau^+ \nu_\tau) \times \mathscr{B}(\tau^+ \rightarrow \pi^+ \pi^- \pi^+ \bar{\nu_\tau})$ + $N(Z^0) \times \mathscr{B}(Z^0 \rightarrow b\bar{b})$

The best purity for a target of 4000 sig events is equal to = 0.4414495369842852
It is associated to a MVA1 cut = 0.9998499999999995 and a MVA2 cut = 0.9986499999999996
Total number of signal = 6862

cut1_min = 0.999, cut1_max = 9, cut2_min = 0.998, cut2_max = 8, cut_step=0.000005.
The best purity for a target of 4000 sig events is equal to = 0.7728891548557775
It is associated to a MVA1 cut = 0.9998650000000057 and a MVA2 cut = 0.9986700000000044
Total number of signal = 6016

The best purity for a target of 4000 sig events is equal to = 0.3681102610809521
It is associated to a MVA1 cut = 0.9998499999999995 and a MVA2 cut = 0.9964999999999998
Total number of signal = 4044.7664361216002

The best purity for a target of 4000 sig events is equal to = 0.5258430335310698
It is associated to a MVA1 cut = 0.9998599999999915 and a MVA2 cut = 0.9963399999999939
Total number of signal = 4009.2649482816005